A SVM model for the dataset 

In [1]:
# Import all libraries
from sklearn import model_selection
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn

from sklearn.svm import SVC    
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, cross_val_score,StratifiedKFold
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from sklearn.impute import KNNImputer

from hn.load_data import load_data

In [2]:

data = load_data()
Y = data['label']
del data['label']

sss = model_selection.StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=None)
cv_4fold = model_selection.StratifiedKFold(n_splits=4)

count = 0 
for train_index, test_index in sss.split(data, Y):
    train_scores = []
    test_scores = []
    count += 1
    #print(f'Iteration {count}')
    data_array = pd.DataFrame.from_dict(data) #Needed to be able to index this later 
    split_X_train = data_array.iloc[train_index]
    split_y_train = Y[train_index]
    split_X_test = data_array.iloc[test_index]
    split_y_test = Y[test_index]

    # Loop over the folds
    for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
        X_validation = split_X_train.iloc[validation_index]
        y_validation = split_y_train[validation_index]
        X_train = split_X_train.iloc[training_index]
        y_train = split_y_train[training_index]


In [3]:
# 1. Scaling (standard, min max, robust)
from sklearn.model_selection import GridSearchCV, cross_val_score,StratifiedKFold  
scaler = preprocessing.StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

# Apply a grid search to find best of three hyperparameters; kernel, gamma and C. Before feature selection 
grid_param =[{'kernel': ('linear','rbf'), 'gamma': [1e-04, 1e-03, 1e-02, 0.1, 1, 10],'C': [1e-04, 1e-03, 1e-02, 0.1, 1, 10]}]
grid_search=GridSearchCV(SVC(),grid_param,n_jobs=-1,verbose=2)
grid_search.fit(X_train_scaled, y_train)

best_hyperparameters = grid_search.best_params_  
print(best_hyperparameters) 
C = best_hyperparameters.get('C')
gamma = best_hyperparameters.get('gamma')
kernel = best_hyperparameters.get('kernel')
best_result = grid_search.best_score_  
print(best_result) 






Fitting 5 folds for each of 72 candidates, totalling 360 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.7175824175824175
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    3.6s finished


In [37]:
model=svm.SVC(kernel=kernel, C=C, gamma=gamma)
model.fit(X_train_scaled, y_train)
#Calculate all class predictions 
Model_predictions_test=model.predict(X_validation)
print(Model_predictions_test)
accuracy=metrics.accuracy_score(y_validation, Model_predictions_test)
print(accuracy)





['T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12'
 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12' 'T12']
0.5


In [36]:
svc = svm.SVC(kernel="linear")

# classifications
rfecv = feature_selection.RFECV(
    estimator=svc, step=1, 
    cv=model_selection.StratifiedKFold(4),
    scoring='roc_auc')
rfecv.fit(X_train_scaled, y_train)

regions = pd.Series(rfecv.support_,index = data.columns[:])
selected_features_rfecv = regions[regions==True].index
print(selected_features_rfecv)

Index(['tf_Gabor_0.5A1.57kurt'], dtype='object')


In [10]:
svmlin = SVC(kernel='linear', gamma='scale')
svmrbf = SVC(kernel='rbf', gamma='scale')
svmpoly = SVC(kernel='poly', degree=3, gamma='scale')
clsfs = [svmlin, svmrbf, svmpoly]
for clf in clsfs:
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_validation)
    print(f"Misclassified: {(y_validation != y_pred).sum()} / {X_validation.shape[0]} ")
    


Misclassified: 15 / 22 
Misclassified: 11 / 22 
Misclassified: 11 / 22 


In [15]:
degrees = [1, 3, 5]
coef0s = [0.01, 0.5, 1]
slacks = [0.01, 0.5, 1]

clsfs = list()
for degree in degrees:
    for coef0 in coef0s:
        for slack in slacks:
            clsfs.append(SVC(kernel='poly', degree=degree, coef0=coef0, C=slack, gamma='scale'))

for clf in clsfs:
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_validation)
    print(f"Misclassified: {(y_validation != y_pred).sum()} / {X_validation.shape[0]} ")

print(80*'#')
grid_param =[{'kernel': ('linear','rbf'), 'gamma': [1e-04, 1e-03, 1e-02, 0.1, 1, 10],'C': [1e-04, 1e-03, 1e-02, 0.1, 1, 10]}]
grid_search=GridSearchCV(SVC(),grid_param,n_jobs=-1,verbose=2)
grid_search.fit(X_train_scaled, y_train)

best_hyperparameters = grid_search.best_params_  
print(best_hyperparameters) 
C = best_hyperparameters.get('C')
gamma = best_hyperparameters.get('gamma')
kernel = best_hyperparameters.get('kernel')
best_result = grid_search.best_score_  
print(best_result) 

model=svm.SVC(kernel=kernel, C=C, gamma=gamma)
model.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_validation)
print(f"Misclassified: {(y_validation != y_pred).sum()} / {X_validation.shape[0]} ")


Misclassified: 6 / 22 
Misclassified: 9 / 22 
Misclassified: 8 / 22 
Misclassified: 6 / 22 
Misclassified: 9 / 22 
Misclassified: 8 / 22 
Misclassified: 6 / 22 
Misclassified: 9 / 22 
Misclassified: 8 / 22 
Misclassified: 8 / 22 
Misclassified: 11 / 22 
Misclassified: 11 / 22 
Misclassified: 8 / 22 
Misclassified: 12 / 22 
Misclassified: 14 / 22 
Misclassified: 7 / 22 
Misclassified: 14 / 22 
Misclassified: 15 / 22 
Misclassified: 10 / 22 
Misclassified: 9 / 22 
Misclassified: 10 / 22 
Misclassified: 10 / 22 
Misclassified: 11 / 22 
Misclassified: 15 / 22 
Misclassified: 9 / 22 
Misclassified: 14 / 22 
Misclassified: 14 / 22 
################################################################################
Misclassified: 14 / 22 


In [13]:
clf = SVC(kernel='rbf', gamma=1)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_validation)
print(f"Misclassified rbf normal: {(y_validation != y_pred).sum()} / {X_validation.shape[0]} ")



clf = SVC(kernel=rbf_kernel)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_validation)
print(f"Misclassified rbf normal: {(y_validation != y_pred).sum()} / {X_validation.shape[0]} ")



Misclassified rbf normal: 11 / 22 


NameError: name 'rbf_kernel' is not defined